# Случайные леса
__Суммарное количество баллов: 10__

В этом задании вам предстоит реализовать ансамбль деревьев решений, известный как случайный лес, применить его к публичным данным пользователей социальной сети Вконтакте, и сравнить его эффективность с ансамблем, предоставляемым библиотекой CatBoost.

В результате мы сможем определить, какие подписки пользователей больше всего влияют на определение возраста и пола человека. 

In [1]:
from sklearn.model_selection import train_test_split
import numpy as np
import pandas
import random
import matplotlib.pyplot as plt
import matplotlib
import copy
from catboost import CatBoostClassifier

In [2]:
from task import gini, entropy, gain

### Задание 1 (2 балла)
Random Forest состоит из деревьев решений. Каждое такое дерево строится на одной из выборок, полученных при помощи bagging. Элементы, которые не вошли в новую обучающую выборку, образуют out-of-bag выборку. Кроме того, в каждом узле дерева мы случайным образом выбираем набор из `max_features` и ищем признак для предиката разбиения только в этом наборе.

Сегодня мы будем работать только с бинарными признаками, поэтому нет необходимости выбирать значение признака для разбиения.

#### Методы
`predict(X)` - возвращает предсказанные метки для элементов выборки `X`

#### Параметры конструктора
`X, y` - обучающая выборка и соответствующие ей метки классов. Из нее нужно получить выборку для построения дерева при помощи bagging. Out-of-bag выборку нужно запомнить, она понадобится потом.

`criterion="gini"` - задает критерий, который будет использоваться при построении дерева. Возможные значения: `"gini"`, `"entropy"`.

`max_depth=None` - ограничение глубины дерева. Если `None` - глубина не ограничена

`min_samples_leaf=1` - минимальное количество элементов в каждом листе дерева.

`max_features="auto"` - количество признаков, которые могут использоваться в узле. Если `"auto"` - равно `sqrt(X.shape[1])`

In [3]:
from task import DecisionTree

### Задание 2 (2 балла)
Теперь реализуем сам Random Forest. Идея очень простая: строим `n` деревьев, а затем берем модальное предсказание.

#### Параметры конструктора
`n_estimators` - количество используемых для предсказания деревьев.

Остальное - параметры деревьев.

#### Методы
`fit(X, y)` - строит `n_estimators` деревьев по выборке `X`.

`predict(X)` - для каждого элемента выборки `X` возвращает самый частый класс, который предсказывают для него деревья.

In [4]:
class RandomForestClassifier:
    def __init__(self, criterion="gini", max_depth=None, min_samples_leaf=1, max_features="auto", n_estimators=10):
        raise NotImplementedError()

    def fit(self, X, y):
        raise NotImplementedError()
    
    def predict(self, X):
        raise NotImplementedError()

### Задание 3 (2 балла)
Часто хочется понимать, насколько большую роль играет тот или иной признак для предсказания класса объекта. Есть различные способы посчитать его важность. Один из простых способов сделать это для Random Forest - посчитать out-of-bag ошибку предсказания `err_oob`, а затем перемешать значения признака `j` и посчитать ее (`err_oob_j`) еще раз. Оценкой важности признака `j` для одного дерева будет разность `err_oob_j - err_oob`, важность для всего леса считается как среднее значение важности по деревьям.

Реализуйте функцию `feature_importance`, которая принимает на вход Random Forest и возвращает массив, в котором содержится важность для каждого признака.

In [5]:
def feature_importance(rfc):
    raise NotImplementedError()

def most_important_features(importance, names, k=20):
    # Выводит названия k самых важных признаков
    idicies = np.argsort(importance)[::-1][:k]
    return np.array(names)[idicies]

Наконец, пришло время протестировать наше дерево на простом синтетическом наборе данных. В результате точность должна быть примерно равна `1.0`, наибольшее значение важности должно быть у признака с индексом `4`, признаки с индексами `2` и `3`  должны быть одинаково важны, а остальные признаки - не важны совсем.

In [8]:
from task import RandomForestClassifier, feature_importance
def synthetic_dataset(size):
    X = [(np.random.randint(0, 2), np.random.randint(0, 2), i % 6 == 3, 
          i % 6 == 0, i % 3 == 2, np.random.randint(0, 2)) for i in range(size)]
    y = [i % 3 for i in range(size)]
    return np.array(X), np.array(y)

X, y = synthetic_dataset(1000)
rfc = RandomForestClassifier(n_estimators=100)
rfc.fit(X, y)
print("Accuracy:", np.mean(rfc.predict(X) == y))
print("Importance:", feature_importance(rfc))

Accuracy: 1.0
Importance: [-5.31075346e-04  6.14068290e-05  1.66429687e-01  1.68173967e-01
  3.27628813e-01 -9.52000321e-04]


### Задание 4 (3 балла)
Теперь поработаем с реальными данными.

Выборка состоит из публичных анонимизированных данных пользователей социальной сети Вконтакте. Первые два столбца отражают возрастную группу (`zoomer`, `doomer` и `boomer`) и пол (`female`, `male`). Все остальные столбцы являются бинарными признаками, каждый из них определяет, подписан ли пользователь на определенную группу/публичную страницу или нет.\
\
Необходимо обучить два классификатора, один из которых определяет возрастную группу, а второй - пол.\
\
Эксперименты с множеством используемых признаков и подбор гиперпараметров приветствуются. Лес должен строиться за какое-то разумное время.

Оценка:
1. 1 балл за исправно работающий код
2. +1 балл за точность предсказания возростной группы выше 65%
3. +1 балл за точность предсказания пола выше 75%

In [9]:
def read_dataset(path):
    dataframe = pandas.read_csv(path, header=0)
    dataset = dataframe.values.tolist()
    random.shuffle(dataset)
    y_age = [row[0] for row in dataset]
    y_sex = [row[1] for row in dataset]
    X = [row[2:] for row in dataset]
    
    return np.array(X), np.array(y_age), np.array(y_sex), list(dataframe.columns)[2:]

In [10]:
X, y_age, y_sex, features = read_dataset("./vk_train.csv")
X_train, X_test, y_age_train, y_age_test, y_sex_train, y_sex_test = train_test_split(X, y_age, y_sex, train_size=0.9)

#### Возраст

In [11]:
from task import rfc_age

rfc_age.fit(X_train, y_age_train)
print("Accuracy:", np.mean(rfc_age.predict(X_test) == y_age_test))
print("Most important features:")
for i, name in enumerate(most_important_features(feature_importance(rfc_age), features, 20)):
    print(str(i+1) + ".", name)

Accuracy: 0.696923076923077
Most important features:
1. mudakoff
2. rhymes
3. styd.pozor
4. ovsyanochan
5. 4ch
6. dayvinchik
7. rapnewrap
8. iwantyou
9. pravdashowtop
10. reflexia_our_feelings
11. pixel_stickers
12. ne1party
13. girlmeme
14. femalemem
15. top_screens
16. tumblr_vacuum
17. ultrapir
18. memeboizz
19. xfilm
20. webestano


#### Пол

In [12]:
from task import rfc_gender

rfc_gender.fit(X_train, y_sex_train)
print("Accuracy:", np.mean(rfc_gender.predict(X_test) == y_sex_test))
print("Most important features:")
for i, name in enumerate(most_important_features(feature_importance(rfc_gender), features, 20)):
    print(str(i+1) + ".", name)

Accuracy: 0.86
Most important features:
1. 40kg
2. 9o_6o_9o
3. girlmeme
4. mudakoff
5. rapnewrap
6. zerofat
7. reflexia_our_feelings
8. modnailru
9. i_d_t
10. be.beauty
11. 4ch
12. igm
13. sh.cook
14. thesmolny
15. combovine
16. bot_maxim
17. femalemem
18. cook_bon
19. academyofman
20. beauty


### CatBoost
В качестве аьтернативы попробуем CatBoost. 

Устаниовить его можно просто с помощью `pip install catboost`. Туториалы можно найти, например, [здесь](https://catboost.ai/docs/concepts/python-usages-examples.html#multiclassification) и [здесь](https://github.com/catboost/tutorials/blob/master/python_tutorial.ipynb). Главное - не забудьте использовать `loss_function='MultiClass'`.\
\
Сначала протестируйте CatBoost на синтетических данных. Выведите точность и важность признаков.

In [13]:
X, y = synthetic_dataset(1000)
print("Accuracy:", np.mean(None == y))
print("Importance:", None)

Accuracy: 0.0
Importance: None


### Задание 5 (3 балла)
Попробуем применить один из используемых на практике алгоритмов. В этом нам поможет CatBoost. Также, как и реализованный ними RandomForest, применим его для определения пола и возраста пользователей сети Вконтакте, выведите названия наиболее важных признаков так же, как в задании 3.\
\
Эксперименты с множеством используемых признаков и подбор гиперпараметров приветствуются.

Оценка:
1. 1 балл за исправно работающий код
2. +1 балл за точность предсказания возростной группы выше 65%
3. +1 балл за точность предсказания пола выше 75%

In [112]:
X, y_age, y_sex, features = read_dataset("./vk_train.csv")
X_train, X_test, y_age_train, y_age_test, y_sex_train, y_sex_test = train_test_split(X, y_age, y_sex, train_size=0.9)
X_train, X_eval, y_age_train, y_age_eval, y_sex_train, y_sex_eval = train_test_split(X_train, y_age_train, y_sex_train, train_size=0.8)

#### Возраст

In [113]:
model = CatBoostClassifier(iterations=100,
                           learning_rate=1,
                           depth=2)

In [114]:
model.fit(X_train, y_age_train)

0:	learn: 1.0009198	total: 2.84ms	remaining: 281ms
1:	learn: 0.9335778	total: 6.38ms	remaining: 313ms
2:	learn: 0.9010180	total: 9.17ms	remaining: 297ms
3:	learn: 0.8645410	total: 11.7ms	remaining: 280ms
4:	learn: 0.8372708	total: 14.2ms	remaining: 271ms
5:	learn: 0.8155979	total: 16.8ms	remaining: 264ms
6:	learn: 0.8018875	total: 19.3ms	remaining: 256ms
7:	learn: 0.7849176	total: 22.7ms	remaining: 261ms
8:	learn: 0.7737392	total: 25.1ms	remaining: 254ms
9:	learn: 0.7591210	total: 27.6ms	remaining: 249ms
10:	learn: 0.7468652	total: 30.3ms	remaining: 245ms
11:	learn: 0.7384427	total: 32.9ms	remaining: 241ms
12:	learn: 0.7278848	total: 35.7ms	remaining: 239ms
13:	learn: 0.7205185	total: 38.4ms	remaining: 236ms
14:	learn: 0.7086528	total: 41ms	remaining: 233ms
15:	learn: 0.7022280	total: 43.7ms	remaining: 229ms
16:	learn: 0.6966889	total: 46.3ms	remaining: 226ms
17:	learn: 0.6926544	total: 49ms	remaining: 223ms
18:	learn: 0.6891319	total: 52ms	remaining: 222ms
19:	learn: 0.6837721	total: 

In [115]:
print("Accuracy:", np.mean(model.predict(X_test).flatten() == y_age_test))
print("Most important features:")
for i, name in enumerate(most_important_features(model.get_feature_importance(), features, 10)):
    print(str(i+1) + ".", name)

Accuracy: 0.7415384615384616
Most important features:
1. ovsyanochan
2. styd.pozor
3. 4ch
4. tumblr_vacuum
5. mudakoff
6. rem_shkola
7. leprum
8. xfilm
9. rhymes
10. dayvinchik


In [116]:
model.save_model('catboost_model1.dump')

#### Пол

In [117]:
model = CatBoostClassifier(iterations=100,
                           learning_rate=1,
                           depth=2)


In [118]:
model.fit(X_train, y_sex_train)

0:	learn: 0.6134447	total: 2.85ms	remaining: 282ms
1:	learn: 0.5659359	total: 7.11ms	remaining: 348ms
2:	learn: 0.5317213	total: 9.93ms	remaining: 321ms
3:	learn: 0.5090701	total: 12.7ms	remaining: 305ms
4:	learn: 0.4912299	total: 15.6ms	remaining: 297ms
5:	learn: 0.4770998	total: 18.6ms	remaining: 292ms
6:	learn: 0.4585450	total: 21.6ms	remaining: 287ms
7:	learn: 0.4451863	total: 24.1ms	remaining: 277ms
8:	learn: 0.4297550	total: 26.7ms	remaining: 270ms
9:	learn: 0.4214611	total: 29.4ms	remaining: 265ms
10:	learn: 0.4121708	total: 32.1ms	remaining: 260ms
11:	learn: 0.4033340	total: 35.2ms	remaining: 258ms
12:	learn: 0.3954826	total: 37.8ms	remaining: 253ms
13:	learn: 0.3900407	total: 40.2ms	remaining: 247ms
14:	learn: 0.3851885	total: 43ms	remaining: 244ms
15:	learn: 0.3806693	total: 45.7ms	remaining: 240ms
16:	learn: 0.3741227	total: 48.7ms	remaining: 238ms
17:	learn: 0.3662417	total: 51.7ms	remaining: 236ms
18:	learn: 0.3621078	total: 54.6ms	remaining: 233ms
19:	learn: 0.3587387	tot

In [119]:
print("Accuracy:", np.mean(model.predict(X_eval).flatten() == y_sex_eval))
print("Most important features:")
for i, name in enumerate(most_important_features(model.get_feature_importance(), features, 10)):
    print(str(i+1) + ".", name)

Accuracy: 0.8623931623931624
Most important features:
1. 40kg
2. modnailru
3. girlmeme
4. zerofat
5. beauty
6. h.made
7. i_d_t
8. mudakoff
9. cook_good
10. thesmolny


In [120]:
model.save_model('catboost_model2.dump')